## CitiBike json file download

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
from pyspark.sql import SQLContext
# On local machine create a spark contxt
#sc = SparkContext(appName="CitiBike_download")
sqlContext = SQLContext(sc)

In [3]:
import requests
import json
import pandas as pd

#### Step 1 : Download four files first

In [4]:
fileNameList = ['station_information','station_status','system_alerts','system_regions']  

In [5]:
# function to clean the Json
def jSonCleaner(strData):
    strData = r.text[r.text.find("["):-2]
    return strData

In [6]:
cleanedData = []

for nfile in fileNameList:
    #removing unwanted string from the starting of the json file and from starting    
    #reading the json file using requests
    r = requests.get("https://gbfs.citibikenyc.com/gbfs/en/" +nfile + ".json")
    # correct json after cleaning
    cleanedData.append(jSonCleaner(r.text)) 

In [39]:
pdf_station_info= pd.DataFrame(json.loads(cleanedData[0]))
pdf_station_status= pd.DataFrame(json.loads(cleanedData[1]))
pdf_system_alerts= pd.DataFrame(json.loads(cleanedData[2]))
pdf_system_regions= pd.DataFrame(json.loads(cleanedData[3]))

pdf_system_alerts

In [70]:
columns = ['alert_id','last_updated', 'station_id','summary','type']
new_pdf_system_alerts = pd.DataFrame(columns=columns)
for index, row in pdf_system_alerts.iterrows():
    if len(row.station_ids)==1:
        new_pdf_system_alerts.loc[len(new_pdf_system_alerts)]=[row['alert_id'],row['last_updated'],row['station_ids'][0],row['summary'],row['type']] 
    else:
        for val in row.station_ids[:]:
            new_pdf_system_alerts.loc[len(new_pdf_system_alerts)]=[row['alert_id'],row['last_updated'],val,row['summary'],row['type']]
        
new_pdf_system_alerts

,alert_id,last_updated,station_id,summary,type
0,6,1511872985,402,Valet service available,OTHER
1,8,1511873002,304,Valet service available,OTHER
2,13,1511866463,3255,Valet service available,OTHER
3,15,1511866841,3236,Valet service available,OTHER
4,46,1511873369,426,Valet service available,OTHER
5,49,1511886262,3443,Valet full,OTHER
6,58,1511873115,281,Valet service available,OTHER
7,79,1511873136,484,Valet service available,OTHER
8,85,1511873330,3223,Valet service available,OTHER
9,86,1511873741,377,Valet service available,OTHER


In [12]:
len(pdf_system_alerts)

0

In [13]:

#Below code works with spark 2.0, it may fail with spark 2.1
sqldf_station_info=sqlContext.createDataFrame(pdf_station_info)   
sqldf_station_status=sqlContext.createDataFrame(pdf_station_status)
try:
    sqldf_system_alerts=sqlContext.createDataFrame(new_pdf_system_alerts)
except:
    sqldf_system_alerts=[]
    
try:
    sqldf_system_regions=sqlContext.createDataFrame(pdf_system_regions)
except:
    sqldf_system_regions=[]
    
    

    
    
    